In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn.python import SKCompat
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from sklearn.utils import shuffle

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# Import data
COLUMNS = ["media_type", "mode_name", "print_height",
           "print_width","print_id", "R", "G", "B","width","height","w/h","size","print_time_sec"]
FEATURES = ["media_type", "media_waste", "mode_name", "print_height",
           "print_width","print_id", "R", "G", "B","width","height","w/h","size"]
LABEL = "print_time_sec"

data_set = pd.read_csv("data_sheet.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS).astype(np.float32)

In [4]:
dummy_fields = ['media_type', 'mode_name','print_id']
for each in dummy_fields:
    dummies = pd.get_dummies(data_set[each], prefix=each, drop_first=False)
    data_set = pd.concat([data_set, dummies], axis=1)
    
fields_to_drop = ['media_type', 'mode_name','print_id']
data = data_set.drop(fields_to_drop, axis=1)
data.to_csv('out.csv')

In [5]:
time = data_set['print_time_sec']
data = data.drop('print_time_sec', axis=1)
data = pd.concat([data, time], axis=1)
data.to_csv('out.csv')
data.shape

(21672, 24)

In [6]:
FEATURES =['print_height', 'print_width', 'R', 'G', 'B', 'width', 'height',
       'w/h', 'size', 'media_type_1.0', 'media_type_2.0',
       'media_type_3.0', 'media_type_4.0', 'media_type_5.0',
       'mode_name_1.0', 'mode_name_2.0', 'mode_name_3.0', 'mode_name_4.0',
       'print_id_1.0', 'print_id_2.0', 'print_id_3.0', 'print_id_4.0',
       'print_id_5.0']

In [22]:
#cut data to three sets
def train_split(data):
    data = shuffle(data)
    total=data.shape[0]
    # split data  to test data 5%
    test = data[-int(0.05*total):]
    # Now remove the test data from the data set 
    data = data[:-int(0.05*total)]
    # split data  to valid data 2%
    valid = data[-int(0.01*total):]
    # Now remove the test data from the data set 
    data = data[:-int(0.01*total)]
    
    return data,test,valid

train,test,valid=train_split(data)

In [8]:
print (valid.shape,test.shape,train.shape)
train.to_csv('train.csv')
test.to_csv('test.csv')
valid.to_csv('valid.csv')

(20373, 24) (1083, 24) (216, 24)


In [9]:
def model_fn(features, targets, mode, params):
  """Model function for Estimator."""

  """input_layer = tf.contrib.layers.fully_connected(features, 
                num_outputs=10,
                weights_initializer = tf.contrib.layers.xavier_initializer(uniform=True),
                weights_regularizer = tf.contrib.layers.l2_regularizer(scale=params["reg"]),
                activation_fn = tf.nn.relu)
    
  #dropout = tf.layers.dropout(inputs=input_layer, rate=0.3, training=mode == learn.ModeKeys.TRAIN)

  output_layer = tf.contrib.layers.fully_connected(input_layer, 
                num_outputs=1,
                weights_initializer = tf.contrib.layers.xavier_initializer(uniform=True),
                weights_regularizer = tf.contrib.layers.l1_regularizer(scale=params["reg"],),
                activation_fn = tf.nn.relu)"""

  # Connect the first hidden layer to input layer
  # (features) with relu activation
    
  first_hidden_layer = tf.contrib.layers.relu(features, 10)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.contrib.layers.linear(first_hidden_layer, 1)


  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"time": predictions}

  # Calculate loss using mean squared error
  loss = tf.losses.mean_squared_error(targets, predictions)

  # Calculate root mean squared error as additional eval metric
  eval_metric_ops = {
      "rmse":
          tf.metrics.root_mean_squared_error(
              tf.cast(targets, tf.float64), predictions)
  }

  train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=params["learning_rate"],
      optimizer="SGD")

  return model_fn_lib.ModelFnOps(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops)

In [10]:
LEARNING_RATE=0.8
REGULIZATION = 2.0
# Set model params
model_params = {"learning_rate": LEARNING_RATE,"reg": REGULIZATION }

# Instantiate Estimator
nn = tf.contrib.learn.Estimator(model_fn=model_fn, params=model_params,model_dir='estimator')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6a2bff1c10>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [11]:
def train_fn(data_set):
    #data_set = (data_set - data_set.mean()) / (data_set.max() - data_set.min())
    feature_cols = data_set[FEATURES]
    labels = data_set[LABEL]
    return feature_cols, labels
def test_fn(data_set):
    feature_cols = data_set[FEATURES]
    labels = data_set[LABEL]
    return feature_cols, labels
def valid_fn(data_set):
    feature_cols = data_set[FEATURES]
    labels = data_set[LABEL]
    return feature_cols, labels

train_x, train_y = train_fn(train)
test_x, test_y = test_fn(test)
valid_x, valid_y = valid_fn(valid)

In [12]:
# Score accuracy
nn.fit(train_x,train_y, steps=30)
#print("Loss: %s" % ev["loss"])
#print("Root Mean Squared Error: %s" % ev["rmse"])

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from estimator/model.ckpt-23142
INFO:tensorflow:Saving checkpoints for 23143 into estimator/model.ckpt.
INFO:tensorflow:loss = 2435.87, step = 23143
INFO:tensorflow:Saving checkpoints for 23172 into estimator/model.ckpt.
INFO:tensorflow:Loss for final step: 2435.86.


Estimator(params={'learning_rate': 0.8, 'reg': 2.0})

In [13]:
train_x.head

<bound method DataFrame.head of        print_height  print_width            R            G            B  \
13311    659.895325  1499.987793   21174224.0   17297712.0   19396472.0   
942      640.088379  1449.986816   30400158.0   28718032.0   23740364.0   
5370     480.059998  1499.996948   10404526.0    9653230.0    7966740.0   
15707    711.115906  1499.987793    6714039.0   10009956.0   15385926.0   
16991   1050.020874  1449.986816   20634398.0   20547738.0   20184982.0   
18333    227.758926  1599.989746   19225568.0   18328248.0   17922728.0   
5576    1349.925293  1499.996948    1365198.0     724980.0     877170.0   
6021    1020.079163  1449.986816   19760020.0   17914426.0   16893802.0   
709      673.772827  1500.029785    8742790.0   15252601.0   16148909.0   
5219     266.405640  1599.989746  157468352.0  152296480.0  149620304.0   
10482    470.915985  1499.996948   13807116.0   17395080.0   22409622.0   
21495   1000.082642  1499.996948   15241402.0   13842440.0   1275833

In [14]:
train_x.iloc[0:3]

,print_height,print_width,R,G,B,width,height,w/h,size,media_type_1.0,...,media_type_5.0,mode_name_1.0,mode_name_2.0,mode_name_3.0,mode_name_4.0,print_id_1.0,print_id_2.0,print_id_3.0,print_id_4.0,print_id_5.0
13311,659.895325,1499.987793,21174224.0,17297712.0,19396472.0,243.0,299.0,0.812709,31.0,0,...,0,0,1,0,0,0,1,0,0,0
942,640.088379,1449.986816,30400158.0,28718032.0,23740364.0,271.0,299.0,0.906355,17.0,0,...,0,0,1,0,0,0,1,0,0,0
5370,480.059998,1499.996948,10404526.0,9653230.0,7966740.0,299.0,188.0,1.590426,21.0,1,...,0,1,0,0,0,1,0,0,0,0


In [15]:
# Score accuracy
ev = nn.evaluate(test_x,test_y, steps=1)
print("Loss: %s" % ev["loss"])
print("Root Mean Squared Error: %s" % ev["rmse"])

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Starting evaluation at 2017-07-31-13:10:58
INFO:tensorflow:Restoring parameters from estimator/model.ckpt-23172
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-07-31-13:10:59
INFO:tensorflow:Saving dict for global step 23172: global_step = 23172, loss = 2182.48, rmse = 46.717
Loss: 2182.48
Root Mean Squared Error: 46.717


In [16]:
# Print out predictions
predictions = nn.predict(valid_x, as_iterable=True)
for i, p in enumerate(predictions):
  print("Prediction %s: %s" % (i + 1, p))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from estimator/model.ckpt-23172
Prediction 1: {'time': 67.013885727007718}
Prediction 2: {'time': 67.013885727007718}
Prediction 3: {'time': 67.013885727007718}
Prediction 4: {'time': 67.013885727007718}
Prediction 5: {'time': 67.013885727007718}
Prediction 6: {'time': 67.013885727007718}
Prediction 7: {'time': 67.013885727007718}
Prediction 8: {'time': 67.013885727007718}
Prediction 9: {'time': 67.013885727007718}
Prediction 10: {'time': 67.013885727007718}
Prediction 11: {'time': 67.013885727007718}
Prediction 12: {'time': 67.013885727007718}
Prediction 13: {'time': 67.013885727007718}
Prediction 14: {'time': 67.013885727007718}
Prediction 15: {'tim

Prediction 1057: {'time': 67.013885727007718}
Prediction 1058: {'time': 67.013885727007718}
Prediction 1059: {'time': 67.013885727007718}
Prediction 1060: {'time': 67.013885727007718}
Prediction 1061: {'time': 67.013885727007718}
Prediction 1062: {'time': 67.013885727007718}
Prediction 1063: {'time': 67.013885727007718}
Prediction 1064: {'time': 67.013885727007718}
Prediction 1065: {'time': 67.013885727007718}
Prediction 1066: {'time': 67.013885727007718}
Prediction 1067: {'time': 67.013885727007718}
Prediction 1068: {'time': 67.013885727007718}
Prediction 1069: {'time': 67.013885727007718}
Prediction 1070: {'time': 67.013885727007718}
Prediction 1071: {'time': 67.013885727007718}
Prediction 1072: {'time': 67.013885727007718}
Prediction 1073: {'time': 67.013885727007718}
Prediction 1074: {'time': 67.013885727007718}
Prediction 1075: {'time': 67.013885727007718}
Prediction 1076: {'time': 67.013885727007718}
Prediction 1077: {'time': 67.013885727007718}
Prediction 1078: {'time': 67.01388

Prediction 2999: {'time': 67.013885727007718}
Prediction 3000: {'time': 67.013885727007718}
Prediction 3001: {'time': 67.013885727007718}
Prediction 3002: {'time': 67.013885727007718}
Prediction 3003: {'time': 67.013885727007718}
Prediction 3004: {'time': 67.013885727007718}
Prediction 3005: {'time': 67.013885727007718}
Prediction 3006: {'time': 67.013885727007718}
Prediction 3007: {'time': 67.013885727007718}
Prediction 3008: {'time': 67.013885727007718}
Prediction 3009: {'time': 67.013885727007718}
Prediction 3010: {'time': 67.013885727007718}
Prediction 3011: {'time': 67.013885727007718}
Prediction 3012: {'time': 67.013885727007718}
Prediction 3013: {'time': 67.013885727007718}
Prediction 3014: {'time': 67.013885727007718}
Prediction 3015: {'time': 67.013885727007718}
Prediction 3016: {'time': 67.013885727007718}
Prediction 3017: {'time': 67.013885727007718}
Prediction 3018: {'time': 67.013885727007718}
Prediction 3019: {'time': 67.013885727007718}
Prediction 3020: {'time': 67.01388

Prediction 4954: {'time': 67.013885727007718}
Prediction 4955: {'time': 67.013885727007718}
Prediction 4956: {'time': 67.013885727007718}
Prediction 4957: {'time': 67.013885727007718}
Prediction 4958: {'time': 67.013885727007718}
Prediction 4959: {'time': 67.013885727007718}
Prediction 4960: {'time': 67.013885727007718}
Prediction 4961: {'time': 67.013885727007718}
Prediction 4962: {'time': 67.013885727007718}
Prediction 4963: {'time': 67.013885727007718}
Prediction 4964: {'time': 67.013885727007718}
Prediction 4965: {'time': 67.013885727007718}
Prediction 4966: {'time': 67.013885727007718}
Prediction 4967: {'time': 67.013885727007718}
Prediction 4968: {'time': 67.013885727007718}
Prediction 4969: {'time': 67.013885727007718}
Prediction 4970: {'time': 67.013885727007718}
Prediction 4971: {'time': 67.013885727007718}
Prediction 4972: {'time': 67.013885727007718}
Prediction 4973: {'time': 67.013885727007718}
Prediction 4974: {'time': 67.013885727007718}
Prediction 4975: {'time': 67.01388

Prediction 7139: {'time': 67.013885727007718}
Prediction 7140: {'time': 67.013885727007718}
Prediction 7141: {'time': 67.013885727007718}
Prediction 7142: {'time': 67.013885727007718}
Prediction 7143: {'time': 67.013885727007718}
Prediction 7144: {'time': 67.013885727007718}
Prediction 7145: {'time': 67.013885727007718}
Prediction 7146: {'time': 67.013885727007718}
Prediction 7147: {'time': 67.013885727007718}
Prediction 7148: {'time': 67.013885727007718}
Prediction 7149: {'time': 67.013885727007718}
Prediction 7150: {'time': 67.013885727007718}
Prediction 7151: {'time': 67.013885727007718}
Prediction 7152: {'time': 67.013885727007718}
Prediction 7153: {'time': 67.013885727007718}
Prediction 7154: {'time': 67.013885727007718}
Prediction 7155: {'time': 67.013885727007718}
Prediction 7156: {'time': 67.013885727007718}
Prediction 7157: {'time': 67.013885727007718}
Prediction 7158: {'time': 67.013885727007718}
Prediction 7159: {'time': 67.013885727007718}
Prediction 7160: {'time': 67.01388

...

Prediction 11341: {'time': 67.013885727007718}
Prediction 11342: {'time': 67.013885727007718}
Prediction 11343: {'time': 67.013885727007718}
Prediction 11344: {'time': 67.013885727007718}
Prediction 11345: {'time': 67.013885727007718}
Prediction 11346: {'time': 67.013885727007718}
Prediction 11347: {'time': 67.013885727007718}
Prediction 11348: {'time': 67.013885727007718}
Prediction 11349: {'time': 67.013885727007718}
Prediction 11350: {'time': 67.013885727007718}
Prediction 11351: {'time': 67.013885727007718}
Prediction 11352: {'time': 67.013885727007718}
Prediction 11353: {'time': 67.013885727007718}
Prediction 11354: {'time': 67.013885727007718}
Prediction 11355: {'time': 67.013885727007718}
Prediction 11356: {'time': 67.013885727007718}
Prediction 11357: {'time': 67.013885727007718}
Prediction 11358: {'time': 67.013885727007718}
Prediction 11359: {'time': 67.013885727007718}
Prediction 11360: {'time': 67.013885727007718}
Prediction 11361: {'time': 67.013885727007718}
Prediction 11

Prediction 13200: {'time': 67.013885727007718}
Prediction 13201: {'time': 67.013885727007718}
Prediction 13202: {'time': 67.013885727007718}
Prediction 13203: {'time': 67.013885727007718}
Prediction 13204: {'time': 67.013885727007718}
Prediction 13205: {'time': 67.013885727007718}
Prediction 13206: {'time': 67.013885727007718}
Prediction 13207: {'time': 67.013885727007718}
Prediction 13208: {'time': 67.013885727007718}
Prediction 13209: {'time': 67.013885727007718}
Prediction 13210: {'time': 67.013885727007718}
Prediction 13211: {'time': 67.013885727007718}
Prediction 13212: {'time': 67.013885727007718}
Prediction 13213: {'time': 67.013885727007718}
Prediction 13214: {'time': 67.013885727007718}
Prediction 13215: {'time': 67.013885727007718}
Prediction 13216: {'time': 67.013885727007718}
Prediction 13217: {'time': 67.013885727007718}
Prediction 13218: {'time': 67.013885727007718}
Prediction 13219: {'time': 67.013885727007718}
Prediction 13220: {'time': 67.013885727007718}
Prediction 13

Prediction 15377: {'time': 67.013885727007718}
Prediction 15378: {'time': 67.013885727007718}
Prediction 15379: {'time': 67.013885727007718}
Prediction 15380: {'time': 67.013885727007718}
Prediction 15381: {'time': 67.013885727007718}
Prediction 15382: {'time': 67.013885727007718}
Prediction 15383: {'time': 67.013885727007718}
Prediction 15384: {'time': 67.013885727007718}
Prediction 15385: {'time': 67.013885727007718}
Prediction 15386: {'time': 67.013885727007718}
Prediction 15387: {'time': 67.013885727007718}
Prediction 15388: {'time': 67.013885727007718}
Prediction 15389: {'time': 67.013885727007718}
Prediction 15390: {'time': 67.013885727007718}
Prediction 15391: {'time': 67.013885727007718}
Prediction 15392: {'time': 67.013885727007718}
Prediction 15393: {'time': 67.013885727007718}
Prediction 15394: {'time': 67.013885727007718}
Prediction 15395: {'time': 67.013885727007718}
Prediction 15396: {'time': 67.013885727007718}
Prediction 15397: {'time': 67.013885727007718}
Prediction 15

Prediction 17356: {'time': 67.013885727007718}
Prediction 17357: {'time': 67.013885727007718}
Prediction 17358: {'time': 67.013885727007718}
Prediction 17359: {'time': 67.013885727007718}
Prediction 17360: {'time': 67.013885727007718}
Prediction 17361: {'time': 67.013885727007718}
Prediction 17362: {'time': 67.013885727007718}
Prediction 17363: {'time': 67.013885727007718}
Prediction 17364: {'time': 67.013885727007718}
Prediction 17365: {'time': 67.013885727007718}
Prediction 17366: {'time': 67.013885727007718}
Prediction 17367: {'time': 67.013885727007718}
Prediction 17368: {'time': 67.013885727007718}
Prediction 17369: {'time': 67.013885727007718}
Prediction 17370: {'time': 67.013885727007718}
Prediction 17371: {'time': 67.013885727007718}
Prediction 17372: {'time': 67.013885727007718}
Prediction 17373: {'time': 67.013885727007718}
Prediction 17374: {'time': 67.013885727007718}
Prediction 17375: {'time': 67.013885727007718}
Prediction 17376: {'time': 67.013885727007718}
Prediction 17

Prediction 19462: {'time': 67.013885727007718}
Prediction 19463: {'time': 67.013885727007718}
Prediction 19464: {'time': 67.013885727007718}
Prediction 19465: {'time': 67.013885727007718}
Prediction 19466: {'time': 67.013885727007718}
Prediction 19467: {'time': 67.013885727007718}
Prediction 19468: {'time': 67.013885727007718}
Prediction 19469: {'time': 67.013885727007718}
Prediction 19470: {'time': 67.013885727007718}
Prediction 19471: {'time': 67.013885727007718}
Prediction 19472: {'time': 67.013885727007718}
Prediction 19473: {'time': 67.013885727007718}
Prediction 19474: {'time': 67.013885727007718}
Prediction 19475: {'time': 67.013885727007718}
Prediction 19476: {'time': 67.013885727007718}
Prediction 19477: {'time': 67.013885727007718}
Prediction 19478: {'time': 67.013885727007718}
Prediction 19479: {'time': 67.013885727007718}
Prediction 19480: {'time': 67.013885727007718}
Prediction 19481: {'time': 67.013885727007718}
Prediction 19482: {'time': 67.013885727007718}
Prediction 19

In [17]:
for i in range(valid_y.shape[0]):
    print(i+1,valid_y.iloc[i])

1 78.0
2 57.0
3 33.0
4 233.0
5 41.0
6 44.0
7 54.0
8 57.0
9 41.0
10 56.0
11 67.0
12 54.0
13 58.0
14 41.0
15 29.0
16 19.0
17 124.0
18 257.0
19 41.0
20 45.0
21 62.0
22 72.0
23 125.0
24 39.0
25 31.0
26 34.0
27 117.0
28 264.0
29 47.0
30 27.0
31 59.0
32 219.0
33 61.0
34 78.0
35 54.0
36 96.0
37 43.0
38 63.0
39 55.0
40 14.0
41 66.0
42 32.0
43 76.0
44 26.0
45 30.0
46 62.0
47 165.0
48 123.0
49 26.0
50 35.0
51 40.0
52 133.0
53 29.0
54 67.0
55 59.0
56 36.0
57 55.0
58 23.0
59 83.0
60 74.0
61 85.0
62 27.0
63 62.0
64 36.0
65 41.0
66 37.0
67 88.0
68 49.0
69 40.0
70 35.0
71 27.0
72 72.0
73 35.0
74 68.0
75 25.0
76 34.0
77 81.0
78 102.0
79 83.0
80 125.0
81 21.0
82 13.0
83 106.0
84 49.0
85 20.0
86 103.0
87 41.0
88 34.0
89 98.0
90 155.0
91 163.0
92 72.0
93 50.0
94 65.0
95 14.0
96 101.0
97 88.0
98 27.0
99 28.0
100 38.0
101 134.0
102 29.0
103 115.0
104 15.0
105 23.0
106 111.0
107 88.0
108 111.0
109 46.0
110 21.0
111 143.0
112 39.0
113 30.0
114 123.0
115 137.0
116 145.0
117 39.0
118 94.0
119 48.0
120 58.0
121

1157 17.0
1158 44.0
1159 176.0
1160 161.0
1161 109.0
1162 247.0
1163 143.0
1164 99.0
1165 79.0
1166 30.0
1167 33.0
1168 19.0
1169 63.0
1170 149.0
1171 78.0
1172 43.0
1173 33.0
1174 40.0
1175 216.0
1176 44.0
1177 47.0
1178 16.0
1179 86.0
1180 20.0
1181 27.0
1182 23.0
1183 31.0
1184 38.0
1185 41.0
1186 124.0
1187 14.0
1188 59.0
1189 46.0
1190 31.0
1191 144.0
1192 85.0
1193 141.0
1194 125.0
1195 55.0
1196 39.0
1197 58.0
1198 79.0
1199 28.0
1200 35.0
1201 113.0
1202 41.0
1203 34.0
1204 30.0
1205 42.0
1206 58.0
1207 103.0
1208 50.0
1209 110.0
1210 28.0
1211 141.0
1212 149.0
1213 49.0
1214 32.0
1215 55.0
1216 24.0
1217 70.0
1218 38.0
1219 33.0
1220 30.0
1221 7.0
1222 29.0
1223 39.0
1224 25.0
1225 38.0
1226 31.0
1227 118.0
1228 50.0
1229 18.0
1230 21.0
1231 30.0
1232 170.0
1233 61.0
1234 41.0
1235 130.0
1236 71.0
1237 76.0
1238 89.0
1239 64.0
1240 114.0
1241 17.0
1242 49.0
1243 108.0
1244 18.0
1245 13.0
1246 17.0
1247 71.0
1248 61.0
1249 37.0
1250 31.0
1251 42.0
1252 15.0
1253 114.0
1254 40.0

3082 87.0
3083 29.0
3084 62.0
3085 79.0
3086 47.0
3087 50.0
3088 30.0
3089 136.0
3090 70.0
3091 22.0
3092 60.0
3093 56.0
3094 105.0
3095 54.0
3096 75.0
3097 45.0
3098 50.0
3099 24.0
3100 21.0
3101 47.0
3102 67.0
3103 23.0
3104 21.0
3105 152.0
3106 61.0
3107 18.0
3108 46.0
3109 175.0
3110 52.0
3111 25.0
3112 53.0
3113 36.0
3114 120.0
3115 58.0
3116 51.0
3117 111.0
3118 34.0
3119 44.0
3120 36.0
3121 34.0
3122 26.0
3123 108.0
3124 88.0
3125 88.0
3126 29.0
3127 31.0
3128 26.0
3129 100.0
3130 126.0
3131 64.0
3132 66.0
3133 118.0
3134 42.0
3135 24.0
3136 159.0
3137 16.0
3138 54.0
3139 59.0
3140 111.0
3141 64.0
3142 45.0
3143 42.0
3144 49.0
3145 208.0
3146 44.0
3147 20.0
3148 42.0
3149 27.0
3150 130.0
3151 119.0
3152 65.0
3153 247.0
3154 35.0
3155 75.0
3156 40.0
3157 37.0
3158 48.0
3159 81.0
3160 45.0
3161 48.0
3162 38.0
3163 46.0
3164 60.0
3165 42.0
3166 26.0
3167 63.0
3168 29.0
3169 39.0
3170 77.0
3171 44.0
3172 29.0
3173 24.0
3174 33.0
3175 25.0
3176 82.0
3177 132.0
3178 64.0
3179 85.0
318

4865 53.0
4866 83.0
4867 38.0
4868 70.0
4869 48.0
4870 28.0
4871 33.0
4872 40.0
4873 24.0
4874 112.0
4875 28.0
4876 93.0
4877 80.0
4878 24.0
4879 11.0
4880 51.0
4881 201.0
4882 37.0
4883 48.0
4884 24.0
4885 105.0
4886 57.0
4887 144.0
4888 52.0
4889 133.0
4890 36.0
4891 46.0
4892 141.0
4893 16.0
4894 29.0
4895 10.0
4896 46.0
4897 47.0
4898 155.0
4899 37.0
4900 21.0
4901 49.0
4902 43.0
4903 41.0
4904 8.0
4905 134.0
4906 50.0
4907 39.0
4908 65.0
4909 30.0
4910 38.0
4911 25.0
4912 70.0
4913 47.0
4914 56.0
4915 126.0
4916 42.0
4917 69.0
4918 43.0
4919 40.0
4920 22.0
4921 45.0
4922 92.0
4923 22.0
4924 67.0
4925 52.0
4926 49.0
4927 34.0
4928 43.0
4929 19.0
4930 70.0
4931 47.0
4932 99.0
4933 46.0
4934 35.0
4935 53.0
4936 47.0
4937 62.0
4938 53.0
4939 126.0
4940 115.0
4941 158.0
4942 17.0
4943 96.0
4944 49.0
4945 57.0
4946 111.0
4947 30.0
4948 89.0
4949 31.0
4950 47.0
4951 32.0
4952 246.0
4953 104.0
4954 111.0
4955 189.0
4956 92.0
4957 24.0
4958 54.0
4959 44.0
4960 30.0
4961 43.0
4962 48.0
4963

6783 65.0
6784 81.0
6785 42.0
6786 33.0
6787 52.0
6788 72.0
6789 52.0
6790 28.0
6791 30.0
6792 26.0
6793 22.0
6794 44.0
6795 150.0
6796 39.0
6797 69.0
6798 31.0
6799 79.0
6800 127.0
6801 35.0
6802 80.0
6803 93.0
6804 62.0
6805 37.0
6806 79.0
6807 27.0
6808 86.0
6809 45.0
6810 38.0
6811 117.0
6812 72.0
6813 124.0
6814 22.0
6815 61.0
6816 164.0
6817 34.0
6818 23.0
6819 41.0
6820 117.0
6821 116.0
6822 93.0
6823 135.0
6824 24.0
6825 24.0
6826 32.0
6827 56.0
6828 35.0
6829 77.0
6830 148.0
6831 37.0
6832 66.0
6833 101.0
6834 87.0
6835 72.0
6836 124.0
6837 38.0
6838 22.0
6839 40.0
6840 61.0
6841 58.0
6842 32.0
6843 44.0
6844 53.0
6845 37.0
6846 68.0
6847 33.0
6848 91.0
6849 36.0
6850 113.0
6851 35.0
6852 78.0
6853 116.0
6854 55.0
6855 174.0
6856 60.0
6857 58.0
6858 137.0
6859 84.0
6860 114.0
6861 270.0
6862 107.0
6863 30.0
6864 59.0
6865 29.0
6866 19.0
6867 52.0
6868 67.0
6869 62.0
6870 48.0
6871 84.0
6872 175.0
6873 57.0
6874 74.0
6875 11.0
6876 33.0
6877 42.0
6878 60.0
6879 56.0
6880 61.0
6

...

12972 99.0
12973 102.0
12974 38.0
12975 111.0
12976 49.0
12977 61.0
12978 113.0
12979 19.0
12980 100.0
12981 151.0
12982 22.0
12983 46.0
12984 76.0
12985 19.0
12986 94.0
12987 62.0
12988 52.0
12989 44.0
12990 42.0
12991 60.0
12992 91.0
12993 72.0
12994 10.0
12995 199.0
12996 89.0
12997 23.0
12998 72.0
12999 29.0
13000 123.0
13001 187.0
13002 81.0
13003 24.0
13004 56.0
13005 15.0
13006 39.0
13007 41.0
13008 26.0
13009 44.0
13010 35.0
13011 32.0
13012 134.0
13013 115.0
13014 42.0
13015 51.0
13016 46.0
13017 45.0
13018 127.0
13019 17.0
13020 22.0
13021 12.0
13022 44.0
13023 98.0
13024 38.0
13025 128.0
13026 29.0
13027 32.0
13028 40.0
13029 56.0
13030 49.0
13031 53.0
13032 48.0
13033 14.0
13034 25.0
13035 25.0
13036 82.0
13037 48.0
13038 53.0
13039 76.0
13040 210.0
13041 108.0
13042 149.0
13043 49.0
13044 44.0
13045 38.0
13046 17.0
13047 27.0
13048 37.0
13049 120.0
13050 94.0
13051 60.0
13052 73.0
13053 233.0
13054 38.0
13055 58.0
13056 31.0
13057 43.0
13058 80.0
13059 86.0
13060 32.0
1306

14824 46.0
14825 67.0
14826 45.0
14827 49.0
14828 101.0
14829 30.0
14830 23.0
14831 43.0
14832 30.0
14833 87.0
14834 79.0
14835 54.0
14836 40.0
14837 158.0
14838 16.0
14839 60.0
14840 19.0
14841 39.0
14842 242.0
14843 101.0
14844 225.0
14845 54.0
14846 22.0
14847 14.0
14848 33.0
14849 23.0
14850 29.0
14851 27.0
14852 43.0
14853 22.0
14854 203.0
14855 62.0
14856 103.0
14857 29.0
14858 55.0
14859 33.0
14860 48.0
14861 80.0
14862 61.0
14863 198.0
14864 34.0
14865 302.0
14866 15.0
14867 101.0
14868 41.0
14869 82.0
14870 22.0
14871 137.0
14872 50.0
14873 182.0
14874 44.0
14875 26.0
14876 50.0
14877 47.0
14878 35.0
14879 63.0
14880 36.0
14881 37.0
14882 34.0
14883 27.0
14884 33.0
14885 25.0
14886 42.0
14887 220.0
14888 46.0
14889 45.0
14890 59.0
14891 49.0
14892 105.0
14893 44.0
14894 60.0
14895 55.0
14896 29.0
14897 41.0
14898 150.0
14899 35.0
14900 25.0
14901 70.0
14902 36.0
14903 45.0
14904 95.0
14905 42.0
14906 101.0
14907 33.0
14908 32.0
14909 37.0
14910 48.0
14911 27.0
14912 38.0
14913

16239 21.0
16240 95.0
16241 145.0
16242 133.0
16243 34.0
16244 30.0
16245 62.0
16246 74.0
16247 34.0
16248 28.0
16249 25.0
16250 47.0
16251 35.0
16252 116.0
16253 104.0
16254 60.0
16255 76.0
16256 122.0
16257 230.0
16258 73.0
16259 105.0
16260 91.0
16261 69.0
16262 110.0
16263 154.0
16264 29.0
16265 38.0
16266 79.0
16267 21.0
16268 57.0
16269 65.0
16270 133.0
16271 25.0
16272 113.0
16273 74.0
16274 53.0
16275 15.0
16276 32.0
16277 85.0
16278 69.0
16279 76.0
16280 43.0
16281 66.0
16282 67.0
16283 58.0
16284 33.0
16285 22.0
16286 28.0
16287 47.0
16288 26.0
16289 82.0
16290 28.0
16291 33.0
16292 24.0
16293 62.0
16294 71.0
16295 13.0
16296 81.0
16297 49.0
16298 39.0
16299 37.0
16300 110.0
16301 247.0
16302 181.0
16303 130.0
16304 104.0
16305 32.0
16306 96.0
16307 171.0
16308 31.0
16309 79.0
16310 153.0
16311 38.0
16312 129.0
16313 37.0
16314 33.0
16315 63.0
16316 41.0
16317 26.0
16318 88.0
16319 40.0
16320 219.0
16321 174.0
16322 14.0
16323 37.0
16324 78.0
16325 26.0
16326 146.0
16327 35.0

18095 46.0
18096 35.0
18097 108.0
18098 90.0
18099 45.0
18100 20.0
18101 81.0
18102 60.0
18103 21.0
18104 60.0
18105 7.0
18106 15.0
18107 45.0
18108 56.0
18109 29.0
18110 51.0
18111 58.0
18112 85.0
18113 23.0
18114 82.0
18115 136.0
18116 105.0
18117 35.0
18118 69.0
18119 63.0
18120 143.0
18121 51.0
18122 41.0
18123 78.0
18124 93.0
18125 144.0
18126 37.0
18127 102.0
18128 195.0
18129 53.0
18130 36.0
18131 45.0
18132 37.0
18133 27.0
18134 20.0
18135 29.0
18136 73.0
18137 45.0
18138 148.0
18139 65.0
18140 64.0
18141 52.0
18142 91.0
18143 106.0
18144 39.0
18145 272.0
18146 67.0
18147 40.0
18148 32.0
18149 79.0
18150 46.0
18151 69.0
18152 67.0
18153 29.0
18154 27.0
18155 65.0
18156 48.0
18157 63.0
18158 178.0
18159 33.0
18160 64.0
18161 27.0
18162 64.0
18163 123.0
18164 56.0
18165 22.0
18166 77.0
18167 75.0
18168 144.0
18169 92.0
18170 50.0
18171 41.0
18172 66.0
18173 44.0
18174 37.0
18175 47.0
18176 31.0
18177 41.0
18178 28.0
18179 111.0
18180 46.0
18181 127.0
18182 23.0
18183 40.0
18184 5

20149 98.0
20150 163.0
20151 33.0
20152 82.0
20153 146.0
20154 31.0
20155 32.0
20156 56.0
20157 46.0
20158 46.0
20159 35.0
20160 68.0
20161 33.0
20162 54.0
20163 22.0
20164 35.0
20165 21.0
20166 53.0
20167 32.0
20168 138.0
20169 124.0
20170 27.0
20171 126.0
20172 27.0
20173 142.0
20174 118.0
20175 36.0
20176 117.0
20177 103.0
20178 35.0
20179 35.0
20180 110.0
20181 31.0
20182 122.0
20183 47.0
20184 29.0
20185 48.0
20186 127.0
20187 45.0
20188 65.0
20189 43.0
20190 42.0
20191 13.0
20192 147.0
20193 32.0
20194 52.0
20195 31.0
20196 13.0
20197 63.0
20198 75.0
20199 49.0
20200 151.0
20201 43.0
20202 73.0
20203 64.0
20204 62.0
20205 108.0
20206 155.0
20207 25.0
20208 49.0
20209 50.0
20210 27.0
20211 35.0
20212 15.0
20213 27.0
20214 61.0
20215 62.0
20216 26.0
20217 92.0
20218 18.0
20219 24.0
20220 54.0
20221 33.0
20222 76.0
20223 80.0
20224 108.0
20225 138.0
20226 61.0
20227 119.0
20228 26.0
20229 87.0
20230 167.0
20231 21.0
20232 42.0
20233 18.0
20234 44.0
20235 104.0
20236 93.0
20237 45.0


In [18]:
type(train_x.values)

numpy.ndarray

In [19]:
type(nn)

tensorflow.contrib.learn.python.learn.estimators.estimator.Estimator

In [24]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

Automatically created module for IPython interactive environment


In [ ]:
# 加载数据集
X = train_x.values
y = train_y.values
nn = SKCompat(nn)
# 用SVM进行学习并记录loss
train_sizes, train_loss, test_loss = learning_curve(SVC(kernel='linear'), X, y, scoring = 'neg_mean_squared_error',
                                                    train_sizes = [0.1, 0.25, 0.5, 0.75, 1])
# 训练误差均值
train_loss_mean = -np.mean(train_loss, axis = 1)
# 测试误差均值
test_loss_mean = -np.mean(test_loss, axis = 1)
# 绘制误差曲线
plt.plot(train_sizes, train_loss_mean, 'o-', color = 'r', label = 'Training')
plt.plot(train_sizes, test_loss_mean, 'o-', color = 'g', label = 'Cross-Validation')
plt.xlabel('Training data size')
plt.ylabel('Loss')
plt.legend(loc = 'best')
plt.show()

/home/hao/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
